In [36]:
import random
from utils import *
from typing import Callable
import sys

In [37]:
#The function that creates randomly the lists for the Set Covering Problem

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]


In [38]:
# A sort of Greedy algorithm:
# This function can be used to reach any ADMISSIBLE solution, if it can find one and if one exists: starting from the first 
# list in the listOfLists (after ordering listOfLists in ascending order of length of the lists), it adds other lists if they 
# contains the missing elements, until it finds a solution or until it explores all the lists (the first list is always 
# selected). NO guarantees on the goodness of the solution: it can be the worst, it can be the best. 
def naive_search(listOfLists, N):
    result= []
    selectedLists=[]
    sortedListOfLists= sorted(listOfLists, key=lambda l: len(l))
    nodes_explored=0
    for l in sortedListOfLists:
        num=0
        nodes_explored+=1
        for i in l:
            if not any(element== i for element in result):
                if num==0:
                    selectedLists.append(l)
                num= num+1
                result.append(i)
                if len(result)== N:
                    w= sum(len(l) for l in selectedLists)
                    return w, nodes_explored
    return None, None


In [39]:
# A sort of depth first, but with some heuristics used to reduce the amount of considered combinations (explored nodes).
# The starting point of the algorithm is the first list of the listOfLists (after ordering listOfLists in ascending order 
# of length of the lists); from this list, the procedure creates the combinations, that, I think, it is more clear to explain 
# with an example. For instance, let's suppose to have 4 lists, already ordered and N=4: l1=[1], l2=[1,2], l3=[1,2],
# l4=[0,1,3]. It starts from l1 and creates as first concatenation l1-l2, then, normally, l1-l2-l3 and l1-l2-l3-l4, but in
# this case l1-l2-l3 does not add new numbers for finding the reached sequence, so it is discarded; thus, the created 
# combination are l1-l2, l1-l2-l4. For every created concatenation, it verifies if this contains the reached sequence, in 
# this case [0,1,2,3]; if so, the concatenation is saved. Subsequently, the algorithm keeps on with l2 and creates l2-l4
# (not l2-l3 and l2-l3-l4, for the reason explained above) and again it verifies if this contains the reached sequence. 
# Finally, the sequence l3-l4 is created. When it has all the selected combination, it chooses the one with the minimum
# weight, in this case l2-l4 or l3-l4, with w=5. I also added to the algorithm a bound to limit the execution time: when
# it has explored more than x number of nodes; it stops and selects the best combination so far.

def depth_first_plus_heuristics(listOfLists, N):
    bestConcatenationSoFar=[]
    sortedListOfLists= sorted(listOfLists, key=lambda l: len(l))
    bound= 50000
    i= 0
    nodes_explored= 0
    for l1 in sortedListOfLists:
        i+= 1
        nodes_explored+= 1
        concatenation= l1
        exit_inner= False
        for l2 in sortedListOfLists[i:]:
            if exit_inner== True:
                break
            nodes_explored+= 1
            if nodes_explored > bound:
                return len(bestConcatenationSoFar), nodes_explored
            for j in l2:
                if not any(element== j for element in concatenation):
                    concatenation+= l2
                    if bestConcatenationSoFar==[]:
                        if contains_the_sequence(concatenation, N):
                            bestConcatenationSoFar= concatenation
                            if len(concatenation)==N:
                                return N, nodes_explored
                    else:
                        if len(concatenation) > len(bestConcatenationSoFar):
                            exit_inner= True
                            break
                        else:
                            if contains_the_sequence(concatenation, N):
                                bestConcatenationSoFar= concatenation
                                if len(concatenation)==N:
                                    return N, nodes_explored
                    break   
                
    return len(bestConcatenationSoFar), nodes_explored


def contains_the_sequence(concatenatedLists, N):
    seq= list(range(N))
    concatenatedLists.sort()
    concatenatedListsWithoutDuplicates= list(dict.fromkeys(concatenatedLists))
    if concatenatedListsWithoutDuplicates == seq:
        return True
    else:
        return False


In [40]:
# Used for the implementation of the A* algorithm, code taken from that of the professor
# Giovanni Squillero for problem 8-puzzle and slightly  modified to be used for this problem

class State:
    def __init__(self):
        self.curr = set()
        self.origin= set()

    def __hash__(self):
        return hash(bytes(self.curr))

    def __eq__(self, other):
        return bytes(self.curr) == bytes(other.curr)

    def __lt__(self, other):
        return bytes(self.curr) < bytes(other.curr)

    def __str__(self):
        return str(self.curr)

    def __repr__(self):
        return repr(self.curr)
    
    
def modify_state(state: State, oldState, newSet): #to change the value of the state during the search
    state.curr= oldState.curr.copy()
    state.origin= newSet.copy()
    state.curr.update(newSet.copy())


In [41]:
# Code taken from that of the professor Giovanni Squillero for problem 8-puzzle and slightly 
# modified to be used for this problem

def search_for_A_star(
    set_of_tuples: set,
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    actual_cost: Callable,
):
    frontier= PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for s in set_of_tuples:
            s= set(s)
            new_state= State()
            modify_state(new_state, state, s)
            cost = actual_cost(s)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p= priority_function(new_state))
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
        if frontier:
            state = frontier.pop()
        else:
            state = None
        
    path= list()
    st= state
    while st.curr !=  set():
        path.append(st.origin)
        st= parent_state[st]
        
    weight= sum(len(s) for s in path)
    
    return len(path), path, weight, len(state_cost)
    
   

In [42]:
n=200

def h(state: State): #estimated cost for the A* algorithm
    temp= list(state.curr) + list(state.origin)
    repetitions= len(temp)-len(set(temp))
    newElements= len(set(temp))- len(state.curr)
    return repetitions - newElements

if __name__ == "__main__":
    
    lists= problem(n, seed= 42) 
    tuples= set(tuple(sorted(l)) for l in lists)
    
    #------------------------------------------------------------------#
    w_naive, nodes_naive= naive_search(tuples, n)
    print("weight_naive= ", w_naive, ", nodes_naive= ", nodes_naive, sep=" ")
   
    #------------------------------------------------------------------#
    # initialState= State()
    # GOAL= State()
    # GOAL.curr= set(range(n))
    # parent_state= dict()
    # state_cost= dict()
    
    # steps_a_star, sets_a_star, w_a_star, nodes_a_star= search_for_A_star(
    #     tuples, 
    #     initialState, 
    #     lambda s: s.curr== GOAL.curr,
    #     parent_state,
    #     state_cost,
    #     priority_function= lambda s: state_cost[s] + h(s),
    #     actual_cost= lambda s: len(s)
    # )
    # print("weight_a_star= ", w_a_star, ", nodes_a_star= ", nodes_a_star, ", steps_a_star= ", steps_a_star, ", sets_a_star= ", sets_a_star, sep=" ")

    #------------------------------------------------------------------#
    w_dfh, nodes_dfh= depth_first_plus_heuristics(lists, n)
    print("weight_dfh= ", w_dfh, ", nodes_dfh= ", nodes_dfh, sep=" ")
    
    
    


weight_naive=  677 , nodes_naive=  26
weight_dfh=  585 , nodes_dfh=  14576
